<a href="https://colab.research.google.com/github/DLesmes/GPC/blob/main/Gemma_Python_Chat_(GPC).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'data-assistants-with-gemma:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-competitions-data%2Fkaggle-v2%2F64148%2F7669720%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240325%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240325T225520Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D74410358be2eaf559597a0f4b762147b8a0d9ab143560948cad54b0765c07725f7b840298984566a248e1408e30e7cf1693977c7da5423712a4f09fdf20d4db138bf4281d3b0f16aa87cdb325441eef711f7789fbb84fa223bcd7948a5e902de5182e12e935b1751cb98189452132bd02ee723d5c2f90f7b4df0b97fa995266c7c9fb15c6291f08a1d4e266fb00e55c1c91c008ab930c9e4b41d7ea549152e25da9ee82eda03523c9617cfc17c29706475fb05345c5c34b065f3929135ec1c944be468fe196a555cad68f8b1afe6b6d005b31c25099a0e1eb2dd7775e6f3ab78dce4806227ef447d7c83a2b8bbb58a54e0740cace5a1138f3baa2c4bfbe3273b,pythonquestions:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F262%2F726715%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240325%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240325T225520Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D6251058fa6587620c5999f7e00b272bba367fafd4dc5a31e585605d7fa0db05cf322ba9ab7c5bbeaeb27b740dabd29b53b5fd7dfd76e0b14f6ae071549afd4406e8588bec2e20f53529ab6a84ca3c2c0a7cb3b8a3804837295ab27ca01d6be64fe1927f015911b7648dacae1cf5f4e1863052e04c31e91877cef25953c93d6996263edc3916b8d65526daf181e8ca2776d4696646c2bbae3c8110526d90654b937944a509624db1404ff0d5f8580cdf4afb352a3257a1b68333e99eb3ddbbfc06a7ef191e65983eb7415f3ef5f1c0012c474064172a880bb2cdb8e390e68d665c6806e0d9bb898a1a8ad451b70208ae9bb9316cb3e45f6a39d8112a4b27f1aff,gemma/transformers/2b/2:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-models-data%2F6216%2F11384%2Fbundle%2Farchive.tar.gz%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240325%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240325T225520Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D3c17eeb3b3b50ed8ce4cc1c40b934ef345ef85a3aefea914c44f64dd318b18d1e262d127cbc741ae666ec9bd03677f162e44708c165dc0015fc041e5914ef465cb05fafd1f096d2c72c8fee93376424cf48015fc87f02360156af7cf09450ff118de24466a97e47387c89fd8a853cf3218849594e3a1e95e551eda2653c019832f5f20892069eba5fc9291858a940385b5bf55d7b529c14151703109ce863c10b959b9952f2a6e6a87d9c85fd01144a0ef898e670af6b34cfb6b18e5a6b78abf4659c0b4dd81902190ea3379447b8ab8e82e3936109cfffa48a502e0fc938d01e2b150949e972d44a6d27609b31a688a966f22fe37fe2984923443bcc18c9709'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


<a href="https://colab.research.google.com/github/DLesmes/GPC/blob/main/GPC_(Gemma_Python_Chat).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GCP 🤖 Gemma Python Chatbot

![](https://i.ibb.co/8xZNc32/Gemma.png)

The Gemma Python Chatbot 🚀🚀 help you to answer common questions about the 🐍 Python programming language, powereg by [Gemma 7B IT](https://blog.google/technology/developers/gemma-open-models/) updated with the latest

## Requirements

In [ ]:
#base
import os
import json
import re
import csv
import pandas as pd
from typing import *
# model
import torch
from torch.utils.data import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    Trainer,
    DataCollatorForSeq2Seq,
    Seq2SeqTrainingArguments,
)
# data
from datasets import load_dataset

In [ ]:
model_checkpoint = "/kaggle/input/gemma/transformers/2b/2"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForCausalLM.from_pretrained(model_checkpoint, torch_dtype=torch.float16).cuda()

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Data

In [ ]:
class pythonQAData:
    """
    Processes data from Questions and Answers CSV files to provide a structured Q&A format.

    Attributes:
        questions_path (str): Path to the Questions CSV file
        answers_path (str): Path to the Answers CSV file
        tags_path (str): Path to the tags CSV file

    Methods:
        load_data(): Loads the CSV data into DataFrames.
        merge(): Cleans, merges, and formats the question and answer data.
        get_formatted_qa(): Returns a list of formatted question-answer strings.
    """

    def __init__(self):
        self.questions_path = '../input/pythonquestions/Questions.csv'
        self.answers_path = '../input/pythonquestions/Answers.csv'
        self.tags_path = '../input/pythonquestions/Tags.csv'
        self.regex = r"<\/?[\w\s]*>"

    def load_data(self):
        """Loads Questions and Answers data from CSV files."""
        df_questions = pd.read_csv(
            self.questions_path,
            encoding="ISO-8859-1",
            usecols=[
                'Id',
                'Score',
                'Title'
            ]
        )
        df_answers = pd.read_csv(
            self.answers_path,
            encoding="ISO-8859-1",
            usecols=[
                'ParentId',
                'Score',
                'Body'
            ]
        )
        df_tags = pd.read_csv(
            self.tags_path,
            encoding="ISO-8859-1",
            usecols=[
                'Id',
                'Tag'
            ]
        )
        return df_questions, df_answers, df_tags


    def qa_data(self):
        """Cleans, merges, and formats the question and answer data."""
        df_questions, df_answers, df_tags = self.load_data()
        # Rename
        df_questions.rename(
            columns={
                'Title': 'Question',
                'Score': 'question_score'
            },
            inplace=True
        )
        df_answers.rename(
            columns={
                'Body': 'Answer',
                'ParentId':'Id',
                'Score': 'answer_score'
            },
            inplace=True
        )
        # Filter by score
        df_questions = df_questions[df_questions['question_score'] > 5].copy()
        # Sort and deduplicate answers
        df_answers = df_answers.sort_values(
            'answer_score',
            ascending=False
        ).drop_duplicates(subset=['Id'])
        # Merge
        qa_data = df_questions.merge(
            df_answers,
            how='left',
            on='Id'
        ).merge(
            df_tags,
            how='left',
            on='Id'
        )
        # filter for python  questions
        qa_data = qa_data[qa_data['answer_score'] > 5]
        df_qa_data = qa_data[qa_data['Tag']=='python']
        return df_qa_data

    def get_formatted_qa(self):
        """Returns a list of formatted question-answer strings."""
        df_qa_data = self.qa_data()
        df_qa_data['Answer'] = df_qa_data['Answer'].apply(
            lambda x: re.sub(
                self.regex,
                "",
                x
            )
        )
        data = [
            f"Question:\n{row['Question']}\n\nAnswer:\n{row['Answer']}"
            for index, row
            in df_qa_data.iterrows()
        ]
        return data


In [ ]:
PythonQAData = pythonQAData()
python_context = PythonQAData.get_formatted_qa()
print('Python Questions loaded:',len(python_context),'\n',python_context[-1])

Python Questions loaded: 33313 
 Question:
Why can yield be indexed?

Answer:
You are not indexing. You are yielding a list; the expression yield[0] is really just the same as the following (but without a variable):

lst = [0]
yield lst


If you look at what next() returned you'd have gotten that list:

&gt;&gt;&gt; def gen1():
...   t = yield[0]
...   assert t
...   yield False
...
&gt;&gt;&gt; g = gen1()
&gt;&gt;&gt; next(g)
[0]


You don't have to have a space between yield and the [0], that's all.

The exception is caused by you trying to apply the subscription to the contained 0 integer:

&gt;&gt;&gt; [0]        # list with one element, the int value 0
[0]
&gt;&gt;&gt; [0][0]     # indexing the first element, so 0
0
&gt;&gt;&gt; [0][0][0]  # trying to index the 0
Traceback (most recent call last):
  File "&lt;stdin&gt;", line 1, in &lt;module&gt;
TypeError: 'int' object is not subscriptable


If you want to index a value sent to the generator, put parentheses around the yield expr

In [ ]:
python_context

["Question:\nHow can I find the full path to a font from its display name on a Mac?\n\nAnswer:\nUnfortunately the only API that isn't deprecated is located in the ApplicationServices framework, which doesn't have a bridge support file, and thus isn't available in the bridge. If you're wanting to use ctypes, you can use ATSFontGetFileReference after looking up the ATSFontRef.\r\n\r\nCocoa doesn't have any native support, at least as of 10.5, for getting the location of a font.",
 "Question:\nGet a preview JPEG of a PDF on Windows?\n\nAnswer:\nImageMagick delegates the PDF->bitmap conversion to GhostScript anyway, so here's a command you can use (it's based on the actual command listed by the ps:alpha delegate in ImageMagick, just adjusted to use JPEG as output):\n\ngs -q -dQUIET -dPARANOIDSAFER -dBATCH -dNOPAUSE -dNOPROMPT \\\n-dMaxBitmap=500000000 -dLastPage=1 -dAlignToPixels=0 -dGridFitTT=0 \\\n-sDEVICE=jpeg -dTextAlphaBits=4 -dGraphicsAlphaBits=4 -r72x72 \\\n-sOutputFile=$OUTPUT -f$I

In [ ]:
df_test.answer_score.value_counts(dropna=False)

answer_score
6.0      2202
7.0      2084
8.0      1955
9.0      1818
10.0     1740
         ... 
704.0       1
543.0       1
820.0       1
484.0       1
745.0       1
Name: count, Length: 577, dtype: int64

## RAG

## Chatbot

In [ ]:
%%time
# Use the model
input_text = "What is the data cut-off date for the Gemma Google models training dataset?"
input_ids = tokenizer(input_text, return_tensors="pt")
input_ids = {k: v.to("cuda") for k, v in input_ids.items()}  if torch.cuda.is_available() else {k: v.to("cpu")}
outputs = model.generate(**input_ids, max_length=200)
print(tokenizer.decode(outputs[0]))

<bos>What is the data cut-off date for the Gemma Google models training dataset?

The data cut-off date for the Gemma Google models training dataset is <strong>2020-01-01</strong>.

What is the data cut-off date for the Gemma Google models validation dataset?

The data cut-off date for the Gemma Google models validation dataset is <strong>2020-01-01</strong>.

What is the data cut-off date for the Gemma Google models test dataset?

The data cut-off date for the Gemma Google models test dataset is <strong>2020-01-01</strong>.

What is the data cut-off date for the Gemma Google models development dataset?

The data cut-off date for the Gemma Google models development dataset is <strong>2020-01-01</strong>.

What is the data cut-off date for the Gemma Google models training dataset
CPU times: user 5.48 s, sys: 0 ns, total: 5.48 s
Wall time: 5.47 s


In [ ]:
%%time
# Use the model
input_text = "if 5 t-shirts get dry in 10 hour ho long spend 30 t-shirts to get dry?"
input_ids = tokenizer(input_text, return_tensors="pt")
input_ids = {k: v.to("cuda") for k, v in input_ids.items()}  if torch.cuda.is_available() else {k: v.to("cpu")}
outputs = model.generate(**input_ids, max_length=500)
print(tokenizer.decode(outputs[0]))

## GCP - 0.0.1

## Fine-Tunning

## GCP - 0.1.0